In [37]:
import pandas as pd
import matplotlib.pyplot as plt

df1 = pd.read_csv('/content/drive/MyDrive/24-1 인공지능/080228-master/실습12_자료/train_SectionDataset.csv',
                  names=['title', 'section'], header=0)
print( df1.shape )
df1 = df1[df1['section'] == 0]
print( df1.shape )
print( df1.head(2) )

(10000, 2)
(2500, 2)
                                      title  section
0  유시민 검찰도 사법도 썩었지 vs 진중권 유시민 망상 대중은 현실로 믿어        0
4                    여야 총선 필승 다짐...국회는 가시밭길        0


In [38]:
df2 = pd.read_csv('/content/drive/MyDrive/24-1 인공지능/080228-master/실습12_자료/test_SectionDataset.csv',
                  names=['title', 'section'], header=0)
df2 = df2[df2['section'] == 0]
print( df2.shape )
print( df2['title'][0] )

new_string = df2['title'][0]
print(new_string)
oneItem = {'title': new_string, 'section': 0}
# df3 = df1.append(oneItem, ignore_index=True)

oneItem_df = pd.DataFrame([oneItem])
df3 = pd.concat([df1, oneItem_df], ignore_index=True)

print(df3.shape)

(44, 2)
박준영 “도자기 집에서 사용” 김선교 “궁궐 살았나”
박준영 “도자기 집에서 사용” 김선교 “궁궐 살았나”
(2501, 2)


In [39]:
# 문자열을 숫자로 바꾸어 벡터화
from sklearn.feature_extraction.text import CountVectorizer

print( df3[['title']].head(3) )
count_vector = CountVectorizer( ngram_range=(1,3) )
c_vector_title = count_vector.fit_transform( df3['title'] )
print( c_vector_title.shape )
print( c_vector_title[0] )
print( "add1: ")
print( c_vector_title[0, 0:40] )  # 배열이라 0 값이 안 찍힘
print( "add2: ")
print( c_vector_title[0, 0], c_vector_title[0, 1],  c_vector_title[0, 33], c_vector_title[0, 255])
print( c_vector_title[1, 0], c_vector_title[1, 1],  c_vector_title[1, 2])

                                      title
0  유시민 검찰도 사법도 썩었지 vs 진중권 유시민 망상 대중은 현실로 믿어
1                    여야 총선 필승 다짐...국회는 가시밭길
2     SBS 여론조사③ 국정지지도 잘한다 49.3 vs 못한다 46.1%
(2501, 21449)
  (0, 14078)	2
  (0, 2208)	1
  (0, 9719)	1
  (0, 12046)	1
  (0, 1119)	1
  (0, 17563)	1
  (0, 6995)	1
  (0, 5896)	1
  (0, 20774)	1
  (0, 7981)	1
  (0, 14083)	1
  (0, 2209)	1
  (0, 9720)	1
  (0, 12047)	1
  (0, 1151)	1
  (0, 17576)	1
  (0, 14089)	1
  (0, 6996)	1
  (0, 5897)	1
  (0, 20775)	1
  (0, 14084)	1
  (0, 2210)	1
  (0, 9721)	1
  (0, 12048)	1
  (0, 1153)	1
  (0, 17578)	1
  (0, 14090)	1
  (0, 6997)	1
  (0, 5898)	1
add1: 

add2: 
0 0 0 0
0 0 0


In [40]:
# 각 행의 단어 묶음 출현 빈도의 코사인 유사도 측정
from sklearn.metrics.pairwise import cosine_similarity

title_c_sim = cosine_similarity( c_vector_title )
print( title_c_sim )

# 가장 높은 인덱스를 가장 위쪽으로 정렬
title_c_sim = title_c_sim.argsort()[:, ::-1]
print( title_c_sim )

[[1.         0.         0.03857584 ... 0.         0.         0.        ]
 [0.         1.         0.         ... 0.         0.06085806 0.        ]
 [0.03857584 0.         1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.06085806 0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]
[[   0  702   17 ... 1652 1651 1250]
 [   1   64 1307 ... 1620 1618    0]
 [   2   56    3 ... 1654 1653 1250]
 ...
 [2498  633 2233 ... 1661 1660    0]
 [2499 1183 1192 ... 1648 1649    0]
 [2500  829  836 ... 1664 1663    0]]


In [41]:
# 최종 추천
def get_recommend_movie_list(df, input_title, top=6):
    # 입력 제목 영화의 인텍스 값 추출
    target_index = df[ df['title'] == input_title].index.values
    print( target_index )

    # 코사인 유사도 중 비슷한 코사인 유사도를 가진 정보를 추출
    sim_index1 = title_c_sim[target_index, :top]
    print( sim_index1 )
    sim_index2 = sim_index1.reshape(-1)
    print( sim_index2 )

    # 본인 제외
    sim_index3 = sim_index2[ sim_index2 != target_index ]
    print( sim_index3 )

    # 최종 선택
    return  df.iloc[ sim_index3 ]

get_recommend_movie_list( df3, new_string )

[2500]
[[2500  829  836  835  834  833]]
[2500  829  836  835  834  833]
[829 836 835 834 833]


,title,section
829,민주평화당 2020년 신년인사회,0
836,트럼프 “김정은 약속 지킬 것”…좋은 관계 재차 강조,0
835,민주평화당 2020년 신년인사회,0
834,민주평화당 2020년 신년인사회,0
833,새해 인사하는 정동영 대표,0
